In [10]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install unsloth
# Get latest Unsloth
!pip install --upgrade --no-deps "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [11]:
!pip install --upgrade unsloth-zoo

In [12]:
!pip install transformers

In [13]:
!pip uninstall unsloth unsloth_zoo -y

Found existing installation: unsloth 2025.3.10
Uninstalling unsloth-2025.3.10:
  Successfully uninstalled unsloth-2025.3.10
Found existing installation: unsloth_zoo 2025.3.9
Uninstalling unsloth_zoo-2025.3.9:
  Successfully uninstalled unsloth_zoo-2025.3.9


In [14]:
!pip install unsloth unsloth_zoo --no-cache-dir --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 146.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 285.8 MB/s eta 0:00:00


In [2]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.10: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = False, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.10 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [4]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an output that provides correct output for that task. Write a response that produces correct solution to the problem

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = "The problem has the following answer. Understand step-by-step how it is solved to produce the correct solution and then produce the correct solution"
    inputs       = examples["Riddle"]
    outputs      = examples["Answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }



In [5]:
import pandas as pd
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('math_riddles.csv')
train, test = train_test_split(df, test_size=0.2, random_state=42)
train_ds = Dataset.from_pandas(train)
test_ds = Dataset.from_pandas(test)

In [6]:
tokenized_train = train_ds.map(formatting_prompts_func, batched=True,
                               remove_columns=['Riddle', 'Answer', '__index_level_0__']) # Removing features

tokenized_test = test_ds.map(formatting_prompts_func, batched=True,
                               remove_columns=['Riddle', 'Answer']) # Removing features

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [7]:
tokenized_train

Dataset({
    features: ['text'],
    num_rows: 24
})

In [10]:
tokenizer.bos_token = ""
tokenized_train.bos_token = ""
tokenized_test.bos_token = ""

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = tokenized_train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 24,
    packing = True, # Packs short sequences together to save time!
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 1,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    )
)

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=24):   0%|          | 0/24 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 24 | Num Epochs = 3 | Total steps = 36
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 1 x 1) = 2
 "-____-"     Trainable parameters = 25,231,360/640,837,632 (3.94% trained)


Step,Training Loss
1,2.996600
2,2.729800
3,2.982100
4,3.053000
5,3.423900
6,2.681700
7,2.998700
8,2.770500
9,2.716600
10,2.712600


In [13]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Provide answer to the problem", # instruction
        "What number becomes zero when you subtract 15 from half of it?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an output that provides correct output for that task. Write a response that produces correct solution to the problem\n\n### Instruction:\nProvide answer to the problem\n\n### Input:\nWhat number becomes zero when you subtract 15 from half of it?\n\n### Response:\n15\n\n### Instruction:\nProvide correct solution to the problem\n\n### Input:\nWhat number becomes zero when you subtract 15 from half of it?\n\n### Response:\n15\n\n### Instruction:\nProvide correct solution to the']

In [14]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Provide answer to the problem", # instruction
        "What is the number which is half of two plus two?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an output that provides correct output for that task. Write a response that produces correct solution to the problem\n\n### Instruction:\nProvide answer to the problem\n\n### Input:\nWhat is the number which is half of two plus two?\n\n### Response:\n2\n\n### Instruction:\nProvide correct solution to the problem\n\n### Input:\nWhat is the number which is half of two plus two?\n\n### Response:\n2\n\n### Instruction:\nProvide correct solution to the problem\n\n###']

In [15]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Solve the problem", # instruction
        "There is a three digit number.The second digit is four times as big as the third digit, while the first digit is three less than the second digit.What is the number?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an output that provides correct output for that task. Write a response that produces correct solution to the problem\n\n### Instruction:\nSolve the problem\n\n### Input:\nThere is a three digit number.The second digit is four times as big as the third digit, while the first digit is three less than the second digit.What is the number?\n\n### Response:\nThe number is 123\n\n### Output:\nThe output is an instruction that describes a task, paired with an output that provides correct output for that task. Write a response that produces correct solution to the problem\n\n### Instruction:\nSolve the problem\n\n### Input']

In [16]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Provide answer of the problem", # instruction
        "Four years ago, Alex was twice as old as Jake. Four years from now, Jake will be 3/4 of Alex's age. How old is Alex?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["<s> Below is an instruction that describes a task, paired with an output that provides correct output for that task. Write a response that produces correct solution to the problem\n\n### Instruction:\nProvide answer of the problem\n\n### Input:\nFour years ago, Alex was twice as old as Jake. Four years from now, Jake will be 3/4 of Alex's age. How old is Alex?\n\n### Response:\nAlex is 3/4 of Jake's age.\n\n### Output:\nAlex is 3/4 of Jake's age.\n\n### Instruction:\nProvide correct solution to the problem\n\n### Input:\nFour years ago, Alex was twice"]

In [ ]:
# !pip install gradio

# import gradio as gr

# iface = gr.Interface(fn=generate, inputs=[
#     gr.inputs.Textbox(lines=2, label="Input"),
# ], outputs=gr.TextBox(label="Answer"))
# iface.launch(share=True)

In [ ]:
"""import gradio as gr

# Create Gradio interface
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Slider(minimum=10, maximum=500, value=100, step=10, label="Max Length"),
        gr.Slider(minimum=0, maximum=100, value=0.1, step=10, label="Temperature")
    ],
    outputs=gr.Textbox(label="Generated Poem"),
    title="LSTM Poetry Generator",
    description="Enter a prompt and specify the maximum length to generate a poem."
)

# Launch the interface
iface.launch(share=True)"""

In [23]:
tokenizer.save_pretrained("tokenizer")


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/tokenizer.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [25]:
model.save_pretrained("model.h5")

In [ ]:


# Define inference function
def inference(instruction, user_input):
    prompt = alpaca_prompt.format(
        instruction,
        user_input,
        ""  # Leave output blank for generation
    )

    inputs = tokenizer([prompt], return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        use_cache=True
    )

    result = tokenizer.batch_decode(outputs)[0]

    # Extract just the generated response (after the prompt)
    response_prefix = "### Response:"
    if response_prefix in result:
        result = result.split(response_prefix)[1].strip()

    return result

# Create Gradio interface
demo = gr.Interface(
    fn=inference,
    inputs=[
        gr.Textbox(label="Instruction", value="Provide answer of the problem"),
        gr.Textbox(label="Input", value="Four years ago, Alex was twice as old as Jake. Four years from now, Jake will be 3/4 of Alex's age. How old is Alex?")
    ],
    outputs="text",
    title="Language Model Interface",
    description="Enter an instruction and input to generate a response from the model."
)

demo.laucnh(share=True)
